# Panorama da desigualdade de gênero no mercado de Inteligência Artificial no Brasil

## Prof. Dr. Neylson Crepalde

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [ ]:
sud = pd.read_csv("../RAIS_VINC_PUB_MG_ES_RJ/RAIS_VINC_PUB_MG_ES_RJ.txt", sep=';', encoding='latin1')
sud.head()

sud.columns

sud["CNAE 2.0 Classe"].dtype

sud = sud.loc[(sud["CNAE 2.0 Classe"] >= 62000) & (sud["CNAE 2.0 Classe"] < 62050) ,:]
sud.shape

sud.to_csv("../data/rais_mg_es_rj.csv", sep="\t", index=False)

del(sud)

In [ ]:
sp = dd.read_csv("../RAIS_VINC_PUB_SP/RAIS_VINC_PUB_SP.txt", sep=';', encoding="latin1", dtype={'CBO Ocupação 2002': 'object'})
sp = sp.loc[(sp["CNAE 2.0 Classe"] >= 62000) & (sp["CNAE 2.0 Classe"] < 62050) ,:]
print(sp.shape)
print(sp.columns)

computed_sp = sp.compute()

computed_sp.shape

computed_sp.to_csv("../data/rais_sp.csv", sep="\t", index=False)

del(computed_sp)

In [2]:
# Dar um merge de linhas nos dois bancos...
sul = pd.concat([
    pd.read_csv("../data/rais_mg_es_rj.csv", sep='\t'),
    pd.read_csv("../data/rais_sp.csv", sep='\t')
], axis=0)

C:\Users\Neylson\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sul.shape

(250621, 62)

In [4]:
sul.columns

Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Regiões Adm DF', 'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Estab', 'Tipo Estab.1', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl 

---

# Análise Exploratória de Dados

## Sexo

In [7]:
print(sul['Sexo Trabalhador'].value_counts())
print(((sul['Sexo Trabalhador'].value_counts() / sul.shape[0] )*100).round(2))

1    167055
2     83566
Name: Sexo Trabalhador, dtype: int64
1    66.66
2    33.34
Name: Sexo Trabalhador, dtype: float64


In [8]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Hora Contrat"])

Faixa Hora Contrat,1,2,3,4,5,6
Sexo Trabalhador,,,,,,
1,2281,36,584,2107,132507,29540
2,1617,44,588,2430,62958,15929


In [9]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Remun Dezem (SM)"])

Faixa Remun Dezem (SM),0,1,2,3,4,5,6,7,8,9,10,11,12,99
Sexo Trabalhador,,,,,,,,,,,,,,
1,50294,252,1081,4907,11549,17772,12618,9715,15323,16623,13429,5685,6139,1668
2,25382,226,1407,6630,7872,9973,6617,4920,6593,5574,3955,1682,1639,1096


In [10]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Remun Média (SM)"])

Faixa Remun Média (SM),0,1,2,3,4,5,6,7,8,9,10,11,99
Sexo Trabalhador,,,,,,,,,,,,,
1,380,1770,10711,19751,26515,17919,13809,21772,22332,16645,6586,6946,1919
2,344,2301,13164,12402,14082,9232,6554,8792,7098,4607,1917,1830,1243


In [20]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Tempo Emprego"])

Faixa Tempo Emprego,1,2,3,4,5,6,7,8,99
Sexo Trabalhador,,,,,,,,,
1,20846,17929,27948,31247,17180,22580,21302,7861,162
2,11746,8154,13771,15452,8804,10873,9991,4684,91


In [22]:
sul.groupby("Sexo Trabalhador").agg({"Idade": "mean"})

,Idade
Sexo Trabalhador,
1,34.004927
2,33.562669
